Le Nutri-Score permet-il de mieux manger? 

Le Nutri-score est un système d'étiquetage nutritionnel à cinq niveaux, allant de A à E et du vert au rouge, placé sur le devant des emballages alimentaires, établi en fonction de la valeur nutritionnelle d'un produit alimentaire. Il a pour but d'aider les consommateurs à reconnaitre la qualité nutritionnelle globale des aliments et les aider à comparer les aliments entre eux, afin de favoriser le choix de produits plus favorable à la santé et ainsi de participer à la lutte contre les maladies chroniques comme les maladies cardiovasculaires, certains cancers, l'obésité et le diabète.

Nous souhaitons ici tenter de retrouver les principaux critères du Nutri-Score en regressant la valeur du nutri-score sur plusieurs variables qualitatives nutritionnelles. 
Puis nous aimerions élargir notre angle d'études en considérant d'autres critères pour quantifier la qualité d'un produit alimentaire (son niveau de transformation et sa provenance notamment). Nous crérons des scoring pour chacune des variables qu'on travaillerait. 
Enfin, nous aimerions mettre en évidence les différents catégories de produits alimentaires en utilisant des algorithmes de clustering à partir de nos scorings. 

Nous travaillerons sur la base de donnée OpenFoodFacts. Open Food Facts est un projet collaboratif dont le but est de constituer une base de données libre et ouverte sur les produits alimentaires commercialisés dans le monde entier. 
La première étape de notre projet est donc de nettoyer cette base de donnée très dense afin de pouvoir commencer nos analyses. 
Nous bornerons notre étude aux produits vendus en France, en ne gardant que les variables qui nous intéressent, pour cela on gardera les produits alimentaires qui auront toutes les variables d'intérêts renseignées.  

On importe les librairies Python qu'on utilisera dans le projet 

In [ ]:
#On importe les modules nécessaires au traitement de la base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#On importe les modules qui seront utilisés lors de la modélisation
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer


On importe la base de donnée OpenFoodFacts

In [ ]:
# Charger le fichier CSV
url_path = 'https://www.data.gouv.fr/fr/datasets/r/164c9e57-32a7-4f5b-8891-26af10f91072'
# Charger le fichier CSV dans un DataFrame pandas
df_openfoodfacts = pd.read_csv(url_path, sep='\t',low_memory=True)  # Assurez-vous de spécifier le bon séparateur s'il est différent de la virgule


On veut connaître ses dimensions avant nettoyage

In [ ]:
#On vérifie que la base est bien chargée
#on affiche 5 lignes aléatoires 
print(df_openfoodfacts.sample(5))

#on veut connaître la taille de la base
print ("Le dataframe compte {} lignes et {} variables".format(df_openfoodfacts.shape[0], df_openfoodfacts.shape[1]))


On commence le nettoyage de la base de donnée OpenFoodFacts: on ne garde que les produits qui sont vendus uniquement en France 

In [ ]:
# Afficher les valeurs uniques de la colonne 'countries_tags' qui sera la variable sur laquelle on va faire un nettoyage 
valeurs_countries_tags = df_openfoodfacts['countries_tags'].unique()
# Afficher les valeurs
print(valeurs_countries_tags)

# Filtrer le DataFrame pour ne conserver que les lignes avec 'en:france' dans la colonne 'countries_tags'
df_france = df_openfoodfacts[df_openfoodfacts['countries_tags'] == 'en:france']
# Afficher les premières lignes du DataFrame résultant
print(df_france.head())

#on veut connaître la taille de la base
print ("Le dataframe compte {} lignes et {} variables".format(df_france.shape[0], df_france.shape[1]))


On souhaite visualiser d'un coup d'oeil les variables d'intérêt de la base de donnée: quelles variables pourrons nous être utiles pour notre analyse? lesquelles sont assez remplies pour nous être utiles? 

In [ ]:
#On calcule le taux de remplissage de chaque variable
def null_factor(df):
  null_rate = ((df.isnull().sum() / df.shape[0])*100).sort_values(ascending=False).reset_index()
  null_rate.columns = ['Variable','Taux_de_Null']
  return null_rate

#Nous alllons désormais commencer à nettoyer la base de données en enlevant les colonnes peu remplis. 
filling_features = null_factor(df_france)
filling_features["Taux_de_Null"] = 100-filling_features["Taux_de_Null"]
filling_features = filling_features.sort_values("Taux_de_Null", ascending=False) 

#Seuil de suppression
sup_threshold = 10

#On affiche le taux de remplissages des variables en fonction d'un seuil de référence
fig = plt.figure(figsize=(20, 35))

font_title = {'family': 'serif',
              'color':  '#114b98',
              'weight': 'bold',
              'size': 18,
             }

sns.barplot(x="Taux_de_Null", y="Variable", data=filling_features, palette="flare")
#Seuil pour suppression des varaibles
plt.axvline(x=sup_threshold, linewidth=2, color = 'r')
plt.text(sup_threshold+2, 65, 'Seuil de suppression des variables', fontsize = 16, color = 'r')

plt.title("Taux de remplissage des variables dans le jeu de données (%)", fontdict=font_title)
plt.xlabel("Taux de remplissage (%)")
plt.show()

On va supprimer les colonnes qui ne sont pas remplies à + de 10% 

In [ ]:
#On ne décide de ne garder que les colonnes remplis à plus de 10%
seuil = 10  
filled_variables = list(filling_features.loc[filling_features['Taux_de_Null'] >= seuil, 'Variable'].values)

#Nouveau Dataset avec les variables conservées
df_cleaned = df_france[filled_variables]

# Affichage du résultat
print ("Le dataframe df_cleaned compte {} lignes et {} variables".format(df_cleaned.shape[0], df_cleaned.shape[1]))

for column_name in df_cleaned.columns:
    print(column_name)

On allège de nouveau la base de donnée en enlevant les variables qui ne nous intéressent pas pour la suite (celles qui contiennent des images des produits notamment par exemple et qui consomment beaucoup de place)

In [ ]:
#On supprime les variables inutiles pour le reste du projet pour alléger la base
useless_columns = [col for col in df_cleaned.columns if 'url' in col or 'image' in col or "categories" in col or "last" in col or "states" in col or "creator" in col ]
df_cleaned = df_cleaned.drop(columns=useless_columns)

print ("Le dataframe df_cleaned compte {} lignes et {} variables".format(df_cleaned.shape[0], df_cleaned.shape[1]))

#On affiche le nom des colonnes restantes
for column_name in df_cleaned.columns:
    print(column_name)


On va maintenant créer une base de donnée unifiée qui ne contient que les variables d'intérêt pour la suite où toutes les occurences pour les scorings créés après sont renseignés 

In [ ]:
# Filtrer les lignes où 'labels_tags' et 'ingredients_text' ne sont pas nulles
df_filtered = df_cleaned.dropna(subset=['labels_tags', 'ingredients_text'])

# Sélectionner les colonnes spécifiques
selected_columns = ['code', 'countries_tags', 'ecoscore_grade', 'nutriscore_grade', 'product_name',
                     'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'proteins_100g',
                     'fat_100g', 'carbohydrates_100g', 'energy-kcal_100g', 'sodium_100g', 'salt_100g',
                     'food_groups_tags', 'labels_tags', 'nutriscore_score', 'nutrition-score-fr_100g',
                     'ecoscore_score', 'ingredients_text', 'nova_group', 'fiber_100g']

# Créer le DataFrame final
df_selected = df_filtered[selected_columns]

# Afficher le DataFrame résultant
df_selected



In [ ]:
print ("Le dataframe df_cleaned compte {} lignes et {} variables".format(df_selected.shape[0], df_selected.shape[1]))


A partir de cette liste, en modifiant à la main, on va garder uniquement les variables que je souhaite étudier pour la suite. 

On va désormais élargir notre analyse en prenant en compte de nouveaux critères qui sont importants lorsqu'on souhaite mieux s'alimenter: le degré de transformation, les additifs et la provenance des produits. On va essayer de créé des variables de scoring pour ces trois catégories avant de faire une comparaison avec le Nutri-Score étudié au dessus. 

1. Analyse du degré de transformation

Pour cette variable, le principal problème est la quantification de la transformation du produit alimentaire. Une approche simple serait de considérer que plus un produit alimentaire contient d'ingrédient, plus il est transformé. 

En s'appuyant sur les critères du nova-score, on va essayer de reproduire son scoring en appliquant les critères décrits ici: https://scanup.fr/degre-de-transformation-des-aliments-la-classification-nova/ .

D'abord, on va donc identifier les produits de la base qui correspondent à des produits naturels (fruits, légumes, poisson qui sont tels quels) en utilisant la catégorie food_groups_tags, avant de faire une analyse des ingrédients sur les produits qu'on identifie comme étant non bruts. 
On observe sur le graphique généré précédemment qu'environ 30% des produits ont la variable ingredient_text renseigné, ce qui nous permettra de générer un nova score sur environ 30% des produits de la base. 
La variable food_groups_tags nous permettra de corriger les cas déviants les plus problématiques après la première étape de scoring. 

In [ ]:
# Afficher les valeurs uniques de la colonne 'main_category_fr'
valeurs_main_category_fr = df_selected['food_groups_tags'].unique()

# Afficher les valeurs
print(valeurs_main_category_fr)


On fait un premier test de scoring en prenant en compte uniquement le nombre d'ingédients listés dans le produit. On veut d'abord regarder comment la variable est renseignée pour choisir le meilleur séparateur d'ingrédiant qui pourra nous permettre de trouver le plus justement possible le nombre d'ingrédents sachant qu'à posteriori, on a vu que la variable n'était pas renseignée de manière homogène. 

In [ ]:
# Afficher les valeurs uniques de la colonne 'main_category_fr'
valeurs_main_category_fr = df_selected['ingredients_text'].unique()

# Afficher les valeurs
print(valeurs_main_category_fr)


On voit que le renseignement de la variable ingredients_text est assez hétérogène: On a donc mis en évidence plusieurs manières de comptabiliser le nb d'ingrédients:
- en comptant le nombre de virgules dans la chaîne de caractère
- en comptant le nombre d'espaces dans la chaîne de caractère (plus fiable mais tend à augumenter le nb d'ingrédient par rapport à la réalité)
- le nombre de points virgule

Finalement, compter les espaces est trop hazardeux, donc on va se concentrer sur le comptage des virgules. On décide donc finalement de compter la somme des virgules et des points virgules puisqu'en général, les deux séparateurs ne sont pas utilisés simultanément, donc compter leur somme nous permettra de balayer le plus de liste d'ingrédients possible tout en faussant le moins possible. 

Il nous restera à traiter ensuite du cas des listes d'ingrédients sans séparateurs. 


In [ ]:
# Compter le nombre d'espaces dans chaque chaîne de caractères de 'ingredients_text'
df_selected['nombre_ingredients'] = df_selected['ingredients_text'].apply(lambda x: x.count(',') + x.count(';'))

# Afficher le DataFrame avec la nouvelle colonne
print(df_selected[['ingredients_text', 'nombre_ingredients']])

# Obtenir un tableau d'occurrences du nombre d'ingrédients
occurrences_nb_ingredients = df_selected['nombre_ingredients'].value_counts()

# Afficher le tableau d'occurrences
print(occurrences_nb_ingredients)

On va établir le scoring suivant: 
- produit brut: 1 si le nombre d'ingrédients est égal à 1 ou 2; 
- ingrédient culiaire: 2 si le nom d'ingrédients est 3 ou 4; 
- produit simplement transformé: 3 si le nombre d'ingrédiants est situé entre 5 et 7 et 
- produit très transformé: 4 si le nombre d'ingrédients est au delà de 8. 

In [ ]:
# Créer une nouvelle colonne 'score' en fonction du nombre d'ingrédients
df_selected['score_transformation'] = df_selected['nombre_ingredients'].apply(lambda n: 1 if n <= 2 else (2 if n <= 4 else (3 if n <= 7 else 4)))

# Afficher le DataFrame avec la nouvelle colonne 'score'
#print(df_selected[['ingredients_text', 'nombre_ingredients', 'score_transformation']])

# Obtenir un tableau d'occurrences du scoring sur la transformation des produits
occurrences_scoring_transformation = df_selected['score_transformation'].value_counts()

# Afficher le tableau d'occurrences
print(occurrences_scoring_transformation)


Pour tester le scoring qu'on vient de créer, nous proposons de regarder les occurences de food_groups_tags qu'on retrouve parmi les produits qu'on a classé en scoring 1 pour la transformation pour voir s'il y a une certaine cohérence (nous ne sommes pas censé retrouver des buscuits par exemple)

In [ ]:
# Filtrer les produits avec un score de 1
score_1_products = df_selected[df_selected['score_transformation'] == 1]

# Afficher les occurrences de food_groups_tags pour les produits avec un score de 1
occurrences_score_1 = score_1_products['food_groups_tags'].value_counts()

# Afficher les résultats
print(occurrences_score_1)


On va vérifier les occurences étranges et voir ce qu'on peut faire pour corriger ces défauts 

In [ ]:
# Filtrer les produits où food_groups_tags prend la valeur "en:sugary-snacks,en:biscuits-and-cakes"
filtered_products = score_1_products[score_1_products['food_groups_tags'] == 'en:sugary-snacks,en:biscuits-and-cakes']

# Afficher la colonne 'product_name' des produits filtrés
print(filtered_products[['product_name', 'ingredients_text']])


Finalement, pour régler le problème des listes d'ingrédients sans séparateurs, on va utiliser la catégorie des produits transformés renseigné dans food_groups_tags pour les mettre directement en score 4, ce qui permettra d'enlever ces passagers clandestins des catégories 1, 2 et 3. 

In [ ]:
# Liste des catégories de produits transformés
categorie_produits_transformes = ['en:composite-foods,en:one-dish-meals',
                                  'en:composite-foods,en:pizza-pies-and-quiches',
                                  'en:sugary-snacks,en:biscuits-and-cakes',
                                  'en:salty-snacks,en:salty-and-fatty-products',
                                  'en:salty-snacks,en:appetizers',
                                  'en:composite-foods,en:sandwiches',
                                  'en:sugary-snacks,en:chocolate-products']

# Mise à jour du score pour les produits dans la liste des catégories de produits transformés
df_selected.loc[df_selected['food_groups_tags'].isin(categorie_produits_transformes), 'score_transformation'] = 4

# Afficher le DataFrame mis à jour
print(df_selected[['product_name', 'food_groups_tags', 'score_transformation']])



On refait un test pour voir si les scorings 1 clandestins ont été corrigés

In [ ]:
# Filtrer les produits avec un score de 1
score_1_products = df_selected[df_selected['score_transformation'] == 1]

# Afficher les occurrences de food_groups_tags pour les produits avec un score de 1
occurrences_score_1 = score_1_products['food_groups_tags'].value_counts()

# Afficher les résultats
print(occurrences_score_1)

2. Origine des aliments 

On va maintenant s'intéresser aux labels des produits alimentaires qu'on considère grâce à la variable labels_tags et tenter de faire une variable de scoring qui nous permettrait de quantifier sa qualité en fonction des labels qu'on lui a accordé quant à sa provenance ou sa production (étiquette bio ou made in France notamment)

In [ ]:
# Afficher les occurrences qui apparaissent plus de 1000 fois dans la colonne 'labels_tags'
occurrences_plus_de_1000 = df_selected['labels_tags'].value_counts()[df_selected['labels_tags'].value_counts() > 1000]

# Afficher le tableau d'occurrences
print(occurrences_plus_de_1000)


On va considérer le scoring suivant pour quantifier l'origine et la provenance des produits alimentaires. 
On se basera sur la provenance (France/UE) et la catégorie de bio 
On définera les scores suivants:
- 1 si le produit est français et bio 
- 2 si le produit est européen et bio 
- 3 si le produit est français
- 4 si le produit est bio 
- 5 sinon 

On a choisi de faire du plus petit au plus grand pour décrire l'évolution du "meilleur" au "pire" pour se caler sur la logique du nutriscore, ce qui nous facilitera les comparaisons par la suite. 

Pour mesurer les différents attributs, on va faire uen analyse dans les chaînes de caractère des labels: s'il y a france ou fr on lui attribuera le fait qu'il est français par exemple. 

In [ ]:
# Créer une nouvelle colonne 'score_labels' initialisée à 0
df_selected['score_labels'] = 0

# Définir des critères et attribuer des scores
critere_bio_france = df_selected['labels_tags'].str.contains('france|fr', case=False) & df_selected['labels_tags'].str.contains('bio|organic', case=False)
critere_eu_bio = df_selected['labels_tags'].str.contains('eu', case=False) & df_selected['labels_tags'].str.contains('bio|organic', case=False)
critere_france = df_selected['labels_tags'].str.contains('france|fr', case=False)
critere_bio = df_selected['labels_tags'].str.contains('bio|organic', case=False)

# Attribuer des scores en fonction des critères (inverser l'ordre)
df_selected.loc[~(critere_bio_france | critere_eu_bio | critere_france | critere_bio) & (df_selected['score_labels'] < 1), 'score_labels'] = 5
df_selected.loc[critere_bio & (df_selected['score_labels'] < 2), 'score_labels'] = 4
df_selected.loc[critere_france & (df_selected['score_labels'] < 3), 'score_labels'] = 3
df_selected.loc[critere_eu_bio & (df_selected['score_labels'] < 4), 'score_labels'] = 2
df_selected.loc[critere_bio_france, 'score_labels'] = 1

# Afficher le DataFrame avec la nouvelle colonne de scoring
print(df_selected[['labels_tags', 'score_labels']])


In [ ]:
df_selected